In [1]:
from preprocessing import load_data

In [2]:
data = load_data()

In [3]:
cuisine_ingredients = {i['cuisine']:}

{'id': 10259,
 'cuisine': 'greek',
 'ingredients': ['romaine lettuce',
  'black olives',
  'grape tomatoes',
  'garlic',
  'pepper',
  'purple onion',
  'seasoning',
  'garbanzo beans',
  'feta cheese crumbles']}